In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from uuid import uuid4
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
from langchain_community.document_loaders import JSONLoader

path = "policy.json"
loader = JSONLoader(file_path=path, jq_schema='.', text_content=False)
docs = loader.load()
print(type(docs))


<class 'list'>


In [2]:
# # PDF 로딩
# path = "data//raw//청년 법령집.pdf"
# loader = PyPDFLoader(file_path=path)
# docs = loader.load()
# print(type(docs))


# for i in range(len(docs)):
#     docs[i].page_content = (docs[i].page_content
#                             .replace('발  간  등  록  번  호','')
#                             .replace('법제처 국가법령정보센터', '')
#                             .replace('법제처', '')
#                             .replace('Korea Law Service Center', '')
#                             .replace('국가법령정보센터', '')
#                             .replace('\n', ' ')
#                             .replace('\t', ' ')
#                             )


<class 'list'>


In [ ]:
# Splitter
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini", 
    chunk_size=500,
    chunk_overlap=100
)
docs = splitter.split_documents(docs)

ids = [str(uuid4()) for _ in range(len(docs))]

# Vector Store
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
COLLECTION_NAME = "youth_policy2"
PERSIST_DIRECTORY = "vector_store/chroma/youth_policy2" #저장할 경로

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    ids=ids,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)


In [4]:
# vector store 연결
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
COLLECTION_NAME = "youth_policy2"
PERSIST_DIRECTORY = "vector_store/chroma/youth_policy2" #저장할 경로

vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

In [5]:
# retriever
retriever = vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={"k":10, "fetch_k":10}
)


# Prompt Template 생성
messages = [
        ("ai", """
        당신은 청년 정책 전문가입니다. 아래 문서의 내용으로만 답변해주세요.
        답변을 모르면 모른다고 대답해주세요.
        {context}"""),
        ("human", "{question}"),
]

prompt_template = ChatPromptTemplate(messages)


# 모델
model = ChatOpenAI(model="gpt-4o-mini")


# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt생성) -> model(정답) -> output parser
chain = {"context":retriever, "question":RunnablePassthrough()} | prompt_template | model | parser


In [6]:
while True:
    query = input()
    if query == '!q':
        break
    print('질문:', query) 
    result = chain.invoke(query)
    print('답변:', result)
    print('='*50)


질문: 울산의 정책 간략히 알려줘
답변 울산의 정책은 다음과 같이 분류할 수 있습니다:

1. **주거 분야**
   - **신축 매입약정형 임대주택 공급사업**: 청년과 신혼부부를 위한 공공주택 공급을 목표로 하며, 울산 전역에 120호의 임대주택을 공급합니다.

2. **일자리 분야**
   - **울산 남구 소상공인 경영주치의(컨설팅)사업**: 소상공인을 위한 맞춤형 컨설팅을 제공하여 경영 문제를 해결합니다.
   - **울산웹툰캠퍼스 운영**: 만화 및 웹툰 작가를 위한 공간과 프로그램을 운영하여 지역 만화산업을 육성합니다.

3. **교육 분야**
   - **고졸인재 성장경로 구축**: 그린모빌리티 관련 교육과정을 운영하고, 직업 교육 협력체계를 구축합니다.
   - **울산콘텐츠코리아랩 운영**: 콘텐츠 창작자를 위한 시설과 프로그램을 제공하여 창작 및 창업을 지원합니다.

4. **참여 및 권리 분야**
   - **울산 중구 청년정책 민·관 협력 활성화**: 청년들이 살고, 놀고, 일할 수 있는 환경을 조성합니다.

이 외에도 다양한 정책들이 시행되고 있으며, 각 정책은 청년과 소상공인, 지역 산업의 발전을 목표로 하고 있습니다.
질문: 부산의 정책 간략히 알려줘
답변 부산에서 시행되는 정책 중 일부는 다음과 같습니다:

1. **울산 보호종료아동 자립수당**: 보호종료 아동에게 매월 50만원의 자립수당을 지급하여 경제적 부담을 완화하고 성공적인 자립을 지원합니다. 지원 기간은 보호 종료 후 최대 5년입니다.

2. **울산 북구 지역 맞춤형 돌봄기능 강화**: 돌봄 수요가 많은 지역에 다함께돌봄센터 및 공동육아나눔터 인프라를 구축하여 방과 후 초등돌봄 공백을 해소하고 아동돌봄의 공공성을 강화합니다.

3. **울산 남구 청년주거급여 분리지급**: 수급가구 내 부모와 떨어져 사는 청년에게 별도로 주거급여를 지급하여 주거 안정성을 높입니다.

4. **울산 매곡산업단지 복합개발사업**: 산업단지 근로자의 주거 불안을 해소하고 청년 창업을 연계할 